In [1]:
import pandas as pd
import time
import requests
import json

In [2]:
from DatabaseCreator import DatabaseCreator

In [3]:
start=time.perf_counter()

In [4]:
import CJDH_local_settings

#Run Database Creator
if __name__ == "__main__":
    db_creator = DatabaseCreator(db_settings=CJDH_local_settings.local_settings['FPL_Points_Predictor'])
    fpl_engine = db_creator.get_engine_for("fpl_data_analysis")

In [5]:
from FPL_League_Data_Scraper import FPL_League_Data_Scraper

# version='2526_gw7'
league_code = "1447854"
season=2526

#Run Data Scraper for TRDL div1 league
if __name__ == "__main__":
    trdl = FPL_League_Data_Scraper(league_code, season)
    picksdf = trdl.scrape_picks()

Data fetched, league name:  Riverside Divisional League 1
Max gameweek data available for is:  13
824703 Tommy Davies GW 1 Pick 1
824703 Tommy Davies GW 1 Pick 2
824703 Tommy Davies GW 1 Pick 3
824703 Tommy Davies GW 1 Pick 4
824703 Tommy Davies GW 1 Pick 5
824703 Tommy Davies GW 1 Pick 6
824703 Tommy Davies GW 1 Pick 7
824703 Tommy Davies GW 1 Pick 8
824703 Tommy Davies GW 1 Pick 9
824703 Tommy Davies GW 1 Pick 10
824703 Tommy Davies GW 1 Pick 11
824703 Tommy Davies GW 1 Pick 12
824703 Tommy Davies GW 1 Pick 13
824703 Tommy Davies GW 1 Pick 14
824703 Tommy Davies GW 1 Pick 15
824703 Tommy Davies GW 2 Pick 1
824703 Tommy Davies GW 2 Pick 2
824703 Tommy Davies GW 2 Pick 3
824703 Tommy Davies GW 2 Pick 4
824703 Tommy Davies GW 2 Pick 5
824703 Tommy Davies GW 2 Pick 6
824703 Tommy Davies GW 2 Pick 7
824703 Tommy Davies GW 2 Pick 8
824703 Tommy Davies GW 2 Pick 9
824703 Tommy Davies GW 2 Pick 10
824703 Tommy Davies GW 2 Pick 11
824703 Tommy Davies GW 2 Pick 12
824703 Tommy Davies GW 2 Pick

In [6]:
#Add a new staging table & data into the database
picks = picksdf[['member', 'memberid','season', 'gw', 'pick', 
                 'element', 'position', 'multiplier', 
                 'is_captain', 'is_vice_captain']]

table_name = "fpl_picks"

if __name__ == "__main__":
    db_creator.create_staging_table_then_insert_data(table_name, data=picks)
    fpl_picks = db_creator.table_to_df(table_name=table_name)
    print(fpl_picks.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1260 entries, 0 to 1259
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   member           1260 non-null   object 
 1   memberid         1260 non-null   float64
 2   season           1260 non-null   float64
 3   gw               1260 non-null   float64
 4   pick             1260 non-null   float64
 5   element          1260 non-null   float64
 6   position         1260 non-null   float64
 7   multiplier       1260 non-null   float64
 8   is_captain       1260 non-null   bool   
 9   is_vice_captain  1260 non-null   bool   
dtypes: bool(2), float64(7), object(1)
memory usage: 81.3+ KB
None


# Get FPL Player current status

In [7]:
#Create DF from FPL API Request
headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
get=requests.get("https://fantasy.premierleague.com/api/bootstrap-static/", headers=headers, timeout=5)
data=json.loads(get.text)

#player_raw = Summary stats for each player for the 2023/24 season
cols=list(data['elements'][0].keys())
player_raw = pd.DataFrame(data['elements'], columns = cols)
player = player_raw[['web_name','id','now_cost','status','news','chance_of_playing_next_round','chance_of_playing_this_round']]
player['season']=20252026
player['datetime_now'] = pd.to_datetime('now')
player.head()

C:\Users\chris\AppData\Local\Temp\ipykernel_21212\2025343833.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player['season']=20252026
C:\Users\chris\AppData\Local\Temp\ipykernel_21212\2025343833.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player['datetime_now'] = pd.to_datetime('now')


,web_name,id,now_cost,status,news,chance_of_playing_next_round,chance_of_playing_this_round,season,datetime_now
0,Raya,1,59,a,,NaN,NaN,20252026,2025-11-22 14:07:50.266243
1,Arrizabalaga,2,42,a,,NaN,NaN,20252026,2025-11-22 14:07:50.266243
2,Hein,3,40,u,Has joined Werder Bremen on loan for the rest ...,0.0,0.0,20252026,2025-11-22 14:07:50.266243
3,Setford,4,40,a,,NaN,NaN,20252026,2025-11-22 14:07:50.266243
4,Gabriel,5,65,i,Thigh injury - Unknown return date,0.0,0.0,20252026,2025-11-22 14:07:50.266243


In [8]:
table_name = "player_status"
if __name__ == "__main__":
    db_creator.create_staging_table_then_insert_data(table_name, data=player)
    player_status = db_creator.table_to_df(table_name=table_name)
    print(player_status.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 752 entries, 0 to 751
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   web_name                      752 non-null    object 
 1   id                            752 non-null    float64
 2   now_cost                      752 non-null    float64
 3   status                        752 non-null    object 
 4   news                          752 non-null    object 
 5   chance_of_playing_next_round  427 non-null    float64
 6   chance_of_playing_this_round  427 non-null    float64
 7   season                        752 non-null    float64
 8   datetime_now                  752 non-null    object 
dtypes: float64(5), object(4)
memory usage: 53.0+ KB
None


In [9]:
# playergw_raw = pd.read_csv(r"data\FPL_Data_GWs_2018to25_vaastav.csv") #df_rest_of_current_season.to_csv('data/FPL_Data_GWs_2018to25_vaastav.csv',index=False)
# [i for i in playergw_raw.columns]

In [10]:
# #Add a new staging table & data into the database
# playergw = playergw_raw[['player_name_id','element','value','season','event','fixture','total_points',
#                          'minutes','goals_scored','assists','team_elo','opp_team_elo','position','bonus','bps',
#                          'clean_sheets','goals_conceded','was_home','expected_assists','expected_goal_involvements',
#                          'expected_goals','expected_goals_conceded','starts',
#                          'cbi','defensive_contribution','recoveries','tackles',
#                          'saves','team_name','opp_team_name']]

# table_name = "playergw"

# if __name__ == "__main__":
#     db_creator.create_staging_table_then_insert_data(table_name, data=playergw)
#     playergwdf = db_creator.table_to_df(table_name=table_name)
#     print(playergwdf.info())

In [11]:
# # Run SELECT and get results
# view = db_creator.run_sql("""SELECT *
#                         FROM playergw
#                         WHERE event = 10
#                         AND season = 20252026
#                         AND team_name = 'Arsenal'
#                         ORDER BY total_points DESC
#                           """)
# view.head()

In [12]:
finish=time.perf_counter()
print(f'Finished in {round(((finish-start)/60),2)} minute(s)')
#10/11/25: Finished in 0.17 minute(s)

Finished in 0.17 minute(s)
